In [1]:
!pip install ollama


In [2]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [4]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short, clean summary. Focus only on meaningful content. "
    "Ignore navigation menus, ads, and repetitive headers. Output in markdown."
)

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled '{website.title}'.\n"
    user_prompt += "Here is the text content of the website. Please summarize it clearly and concisely:\n\n"
    user_prompt += website.text
    return user_prompt

In [1]:
def summarize(url, model="llama3"):
    """
    Given a URL, scrape and summarize it using Ollama (local LLM model).
    """
    website = Website(url)
    messages = messages_for(website)
    
    response = ollama.chat(model=model, messages=messages)
    return response['message']['content']

In [2]:
def display_summary(url, model="llama3"):
    summary = summarize(url, model=model)
    display(Markdown(summary))
